# Advanced Training Pipeline

Once we have set up our pipeline, we can focus on trying out different combinations of data and models. However, to do this, we need to make sure that our pipeline can be easily adjusted for the specific task. This means that we should be able to run the pipeline using different models, settings, and other components. Imagine if we want to run 100 experiments with various combinations, it would be difficult and time-consuming to write separate scripts for each experiment. It would also be challenging to maintain and manage multiple pipelines or configurations if we cannot reuse the same pipeline.

<img src="https://raw.githubusercontent.com/facebookresearch/hydra/master/website/static/img/Hydra-Readme-logo2.svg" alt="logo" width="40%" />

In this section, I will demonstrate how we can create complex configurations and run multiple experiments with just one command using [Hydra](https://github.com/facebookresearch/hydra), a tool developed by Facebook Research. Hydra simplifies the process by providing a convenient way to manage configurations. We will begin by running a basic experiment with the default settings and then proceed to run multiple experiments effortlessly using Hydra. Finally, we will combine Hydra with the [Optuna Framework](https://optuna.org/) to perform a Hyperparameter Sweep, allowing us to run thousands of experiments to find the best configuration for our model.

This section is inspired from [ashleve/lightning-hydra-template](https://github.com/ashleve/lightning-hydra-template). Please read the repository the fully understand Lightning+Hydra setup

In [3]:
!pip install -q hydra-core hydra-colorlog timm

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [22]:
import os

ROOT_DIR = os.path.dirname(os.path.abspath(''))
DATA_DIR = os.path.join(ROOT_DIR, 'data/food-101-tiny')

TRAIN_DATA_PATH = os.path.join(DATA_DIR, 'train')
VAL_DATA_PATH = os.path.join(DATA_DIR, 'valid')

%cd 02_MODEL_DEVELOPMENT/

[Errno 2] No such file or directory: '02_MODEL_DEVELOPMENT/'
/home/haritsahm/Documents/Getting Started/02_Model_Development


## Hydra Configuration Pipeline

In Hydra, you can create separate configuration files for different parts of your project or experiment. Each file contains settings and parameters specific to that part. For example, you might have one file for model configuration, another for dataset configuration, and so on. Hydra allows you to define a hierarchical structure for these configuration files. You can specify relationships and dependencies between them. For instance, you can have a base configuration file that contains common settings shared by all other configurations, and then have specialized configuration files that override or extend the base settings.

When you run your experiment or application, Hydra intelligently combines these configuration files to create a single, cohesive configuration. It merges the settings from different files based on their hierarchy and resolves any conflicts or inconsistencies. The resulting configuration is a composition of all the specified settings, providing a comprehensive configuration that captures the specific requirements of your experiment. This modular approach makes it easy to manage and reuse configurations across different experiments or projects.

For more details on how to use hydra, please read the [documentation](https://hydra.cc/docs/intro/).

We're going to wrap the training pipeline in `train.py`. Basicly the function will construct the pipeline based on the configuration file.

This example shows how to use hydra using python, but the same method also works with `cli`.

### Open a Configurations

In [2]:
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="configs/"):
    cfg = compose(config_name="train.yaml")
    print(OmegaConf.to_yaml(cfg))

work_dir: ${hydra:runtime.cwd}
task_name: train
data_dir: data/
tags:
- dev
train: true
test: true
ckpt_path: null
seed: null
ignore_warnings: true
print_config: true
data:
  _target_: src.dataset.Food101LitDatamodule
  data_dir: ${data_dir}
  input_size:
  - 384
  - 384
  batch_size: 16
  num_workers: 4
  pin_memory: false
model:
  _target_: src.models.ClassificationLightningModule
  num_classes: 10
  lr: 0.0001
  net:
    _target_: src.models.ResNet18
    input_channels: 3
    num_classes: ${..num_classes}
callbacks:
  model_checkpoint:
    _target_: lightning.pytorch.callbacks.ModelCheckpoint
    dirpath: null
    filename: epoch_{epoch:03d}
    monitor: val/acc
    mode: max
    save_last: true
    save_top_k: 1
    auto_insert_metric_name: false
  early_stopping:
    _target_: lightning.pytorch.callbacks.EarlyStopping
    monitor: val/acc
    patience: 100
    mode: max
trainer:
  _target_: lightning.pytorch.trainer.Trainer
  default_root_dir: null
  min_epochs: 1
  max_epochs: 10

The configuration above

### Running an Experiment

Using `hydra` is simple, just run the following command.

Equivalent CLI command:
```
python3 train.py -m experiment=food101
```

In [17]:
from src.training_pipeline import train

with initialize(version_base=None, config_path="configs/"):
    cfg = compose(
        config_name="train.yaml",
        overrides=[f"data_dir={DATA_DIR}", "experiment=food101"]
    )
    train(cfg)

/usr/local/lib/python3.8/dist-packages/lightning/pytorch/utilities/parsing.py:197: UserWarning: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
  rank_zero_warn(
No logger configs found! Skipping...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type               | Params
----------------------------------------------------
0 | net          | ResNet18           | 12.6 M
1 | criterion    | CrossEntropyLoss   | 0     
2 | train_acc    | MulticlassAccuracy | 0     
3 | val_metrics  | MetricCollection   | 0     
4 | test_metrics | MetricCollection   | 0     
----------------------------------------------------
12.6 M    Trainable params
0         Non-trainable params
12.6 M    Total params
50.251    To

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
Restoring states from the checkpoint path at /home/haritsahm/Documents/Getting Started/02_Model_Development/checkpoints/epoch_007.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/haritsahm/Documents/Getting Started/02_Model_Development/checkpoints/epoch_007.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.4359999895095825     │
│        test/auroc         │     0.043674286454916     │
│          test/f1          │    0.4359999895095825     │
│         test/loss         │     1.644534707069397     │
│         test/prec         │    0.4359999895095825     │
│         test/rec          │    0.4359999895095825     │
└───────────────────────────┴───────────────────────────┘

### Running an Experiment with Overriden Parameters

We can override existing configuration to execute it with different configurations.

Equivalent CLI command:
```
python3 train.py -m experiment=food101 model=resnet18 model.lr=0.001 data.batch_size=8
```

In [20]:
overrides = [
    f"data_dir={DATA_DIR}",
    "experiment=food101",
    "model=resnet18",
    "model.lr=0.001",
    "data.batch_size=8",
]

with initialize(version_base=None, config_path="configs/"):
    cfg = compose(
        config_name="train.yaml",
        overrides=overrides,
    )
    train(cfg)

Global seed set to 12345
/usr/local/lib/python3.8/dist-packages/lightning/fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /home/haritsahm/Documents/Getting Started/02_Model_Development/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type               | Params
----------------------------------------------------
0 | net          | ResNet18           | 12.6 M
1 | criterion    | CrossEntropyLoss   | 0     
2 | train_acc    |

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
Restoring states from the checkpoint path at /home/haritsahm/Documents/Getting Started/02_Model_Development/checkpoints/epoch_008.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/haritsahm/Documents/Getting Started/02_Model_Development/checkpoints/epoch_008.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.3240000009536743     │
│        test/auroc         │   0.015833333134651184    │
│          test/f1          │    0.3240000009536743     │
│         test/loss         │    2.2126882076263428     │
│         test/prec         │    0.3240000009536743     │
│         test/rec          │    0.3240000009536743     │
└───────────────────────────┴───────────────────────────┘

In [24]:
overrides = [
    f"data_dir={DATA_DIR}",
    "experiment=food101",
    "model=timm",
    "model.net.model_name=resnetv2_50",
    "model.lr=0.001",
    "data.batch_size=8",
]

with initialize(version_base=None, config_path="configs/"):
    cfg = compose(
        config_name="train.yaml",
        overrides=overrides,
    )
    train(cfg)

Global seed set to 12345
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type               | Params
----------------------------------------------------
0 | net          | ResNetV2           | 23.5 M
1 | criterion    | CrossEntropyLoss   | 0     
2 | train_acc    | MulticlassAccuracy | 0     
3 | val_metrics  | MetricCollection   | 0     
4 | test_metrics | MetricCollection   | 0     
----------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.083    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
Restoring states from the checkpoint path at /home/haritsahm/Documents/Getting Started/02_Model_Development/checkpoints/epoch_009.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/haritsahm/Documents/Getting Started/02_Model_Development/checkpoints/epoch_009.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.3779999911785126     │
│        test/auroc         │   0.020233333110809326    │
│          test/f1          │    0.3779999911785126     │
│         test/loss         │    1.7338745594024658     │
│         test/prec         │    0.3779999911785126     │
│         test/rec          │    0.3779999911785126     │
└───────────────────────────┴───────────────────────────┘

In [23]:
!pip install -q timm

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


### Running Multiple Experiments

'hydra' is capable to run multiple experiments with different configurations. We need to add `--multirun` or `-m` to the python command and execute the main file. <br>
This command will run 4 experiments using `timm` model with lr-bs pairs of: `(0.001,8)`, `(0.001,16)`, `(0.0001,8)`, `(0.0001,16)`.

Please read the [documentation](https://hydra.cc/docs/tutorials/basic/running_your_app/multi-run/).

Unfortunately, `multirun` method is not executable via notebook cell like the previous cells, it must be executed with python command.
From the `02_Model_Development/`, run the following command to run multiple experiments:
```
python3 train.py -m experiment=food101 model=timm trainer.max_epochs=5 model.lr=0.001,0.0001 data.batch_size=8,16 logger=wandb
```

In [41]:
!python3 train.py -m experiment=food101 model=timm trainer.max_epochs=5 model.lr=0.001,0.0001 data.batch_size=8,16 logger=wandb data_dir=../data/food-101-tiny/

[2023-05-28 16:40:54,863][HYDRA] Launching 4 jobs locally
[2023-05-28 16:40:54,863][HYDRA] 	#0 : experiment=food101 model=timm trainer.max_epochs=5 model.lr=0.001 data.batch_size=8 logger=wandb data_dir=../data/food-101-tiny/
Global seed set to 12345
[2023-05-28 16:40:55,010][src.train_pipeline][INFO] - Instantiating datamodule <src.dataset.Food101LitDatamodule>
[2023-05-28 16:40:55,224][src.train_pipeline][INFO] - Instantiating model <src.models.ClassificationLightningModule>
/usr/local/lib/python3.8/dist-packages/lightning/pytorch/utilities/parsing.py:197: UserWarning: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
  rank_zero_warn(
[2023-05-28 16:40:56,272][src.train_pipeline][INFO] - Instantiating callbacks...
[2023-05-28 16:40:56,273][src.utils][INFO] - Instantiating callback <lightning.pytorch.callbacks.ModelCheckpoint>
[2023-05-28 16:40:56,274][src.utils]

## Hyperparameter Sweeps using Optuna

<img src="https://raw.githubusercontent.com/optuna/optuna/master/docs/image/optuna-logo.png" alt="logo" width="40%" />

[Optuna](https://optuna.org/) automates the process of searching for the optimal combination of hyperparameters by intelligently exploring the hyperparameter space. It employs state-of-the-art algorithms, including tree-structured Parzen estimators (TPE), genetic algorithms, and particle swarm optimization, to efficiently navigate the search space and find the best set of hyperparameters.

To use Optuna, you define an objective function that evaluates the performance of your model using a specific set of hyperparameters. Optuna then iteratively samples different hyperparameter configurations, evaluates their performance by calling the objective function, and updates its search strategy based on the collected results. This process continues for a specified number of iterations or until a convergence criterion is met.

To combine the capabilities of Hydra and Optuna, there is an [Optuna Sweeper plugin](https://hydra.cc/docs/plugins/optuna_sweeper/) available for Hydra. This plugin integrates Optuna's hyperparameter optimization capabilities into Hydra's configuration management. With the Optuna Sweeper plugin, you can define a search space for hyperparameters in your Hydra configuration files. During the hyperparameter sweep, Optuna will sample different combinations of hyperparameters and run the experiments accordingly.

By leveraging the Optuna Sweeper plugin in Hydra, you can easily perform hyperparameter optimization and explore different configurations without the need for writing separate scripts or managing multiple pipelines. It simplifies the process of finding the best hyperparameters for your machine learning models.


Install the Optuna Sweeper plugin for Hydra.
```
pip install hydra-optuna-sweeper
```

The following command will run 20 experiments to find the best configuration based on the `val/acc` metric. It will try to find the best `timm` model by experimenting with different `batch_size`, `lr`, and `model_name` parameters.

Unfortunately, `multirun` is not available in notebook.
From the `02_Model_Development/`, run the following command to run multiple experiments:
```
python3 train.py -m experiment=food101 hparams_search=food101_optuna model=timm trainer.max_epochs=5 logger=wandb
```

In [43]:
!pip install -q hydra-optuna-sweeper


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 308 kB 893 kB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 224 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 80 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 618 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 3.5 MB/s eta 0:00:01
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=69a00f0615b0c956b9156a109e78c0d0cc2f69ef9d8dbc5271596eec0749ade0
  Stored in directory: /tmp/pip-ephem-wheel-cache-y7dpn22c/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
You sho

In [ ]:
!python3 train.py -m experiment=food101 hparams_search=food101_optuna model=timm trainer.max_epochs=5 logger=wandb